In [1]:
import sklearn.neighbors._base
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
# !pip install impyute
# !pip install fancyimpute
from sklearn import datasets
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as skLDA
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from scipy import stats
import numpy as np
# import impyute as impy
from fancyimpute import SoftImpute#, MatrixFactorization
import pandas as pd
import time
import math
from missingpy import MissForest
from sklearn.model_selection import train_test_split
from numpy.linalg import norm, inv

from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from fancyimpute import SimpleFill, SoftImpute
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from imblearn.under_sampling import InstanceHardnessThreshold

from imblearn.metrics import geometric_mean_score
from imblearn.metrics import sensitivity_score

from sklearn.metrics import f1_score, roc_auc_score
from sklearn import metrics
from imblearn.datasets import fetch_datasets
from imblearn.over_sampling import KMeansSMOTE
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SVMSMOTE
from functions.OsImp import *
import warnings
warnings.filterwarnings("ignore")

from functions.eval import *
from functions.utils import *
from functions.OsImp import *
from functions.OsImp_star import *
from functions.dpers import *
from functions.dimv import dimv
# imputer = dimv

imputer = IterativeImputer().fit_transform
# imputer = SoftImpute(convergence_threshold=0.1, max_iters=10).fit_transform
# imputer = MissForest().fit_transform

In [2]:
  df = fetch_datasets()['ecoli']
  X, y = df.data, df.target
  le = LabelEncoder()
  y = le.fit_transform(y)
  G = len(np.unique(y))
  print(np.shape(X), G, sum(y==0)*.6*.8, sum(y==1)*.6*.8)
  X[:3]

(336, 7) 2 144.48 16.8


array([[0.49, 0.29, 0.48, 0.5 , 0.56, 0.24, 0.35],
       [0.07, 0.4 , 0.48, 0.5 , 0.54, 0.35, 0.44],
       [0.56, 0.4 , 0.48, 0.5 , 0.49, 0.37, 0.46]])

In [3]:
missing_rates = np.array([0.0, 0.1, 0.2, 0.3, 0.4])
res_all = []
for missing_rate in missing_rates:
  K, nMC, result = 2, 10, []
  n = np.array([-90,80])
  res = np.asarray([get_all_res_star(X, y, i, n, K, G,imputer, missing_rate) for i in range(nMC)], dtype = object)
  res = show_result(res)
  res_all.append(res)    

In [4]:
concat_tab = pd.concat((res_all[0][1][0], res_all[0][1][1], 
                        res_all[1][1][0], res_all[1][1][1], 
                        res_all[2][1][0], res_all[2][1][1], 
                        res_all[3][1][0], res_all[3][1][1], 
                        res_all[4][1][0], res_all[4][1][1]))

In [5]:
leftindex = pd.MultiIndex.from_product([["0%", "10%", "20%", "30%", "40%"],
                                        ["F-1 score","sensitivity"], 
                                        ["SVM", "LR", "DCT"]], 
                                       names = ["missing rate", "metric", "classifier"])
all_res = pd.DataFrame(concat_tab.to_numpy(), index = leftindex, columns = concat_tab.columns)
all_res

NearMiss         kmeanSmt  \
missing rate metric      classifier                                     
0%           F-1 score   SVM           0.54$\pm$0.17  0.916$\pm$0.013   
                         LR          0.756$\pm$0.104   0.92$\pm$0.017   
                         DCT         0.768$\pm$0.115   0.899$\pm$0.03   
             sensitivity SVM         0.476$\pm$0.175   0.91$\pm$0.015   
                         LR           0.707$\pm$0.13  0.913$\pm$0.021   
                         DCT         0.727$\pm$0.136  0.903$\pm$0.026   
10%          F-1 score   SVM         0.415$\pm$0.142  0.913$\pm$0.018   
                         LR          0.641$\pm$0.133  0.918$\pm$0.017   
                         DCT          0.534$\pm$0.23  0.899$\pm$0.024   
             sensitivity SVM         0.349$\pm$0.142  0.901$\pm$0.025   
                         LR          0.571$\pm$0.148  0.908$\pm$0.022   
                         DCT         0.468$\pm$0.245  0.898$\pm$0.024   
20%          F-1 score   SVM          0.38$\pm$0.098  0.893$\pm$0.023   
                         LR          0.623$\pm$0.108  0.894$\pm$0.023   
                         DCT         0.602$\pm$0.231  0.904$\pm$0.024   
             sensitivity SVM         0.312$\pm$0.077  0.877$\pm$0.027   
                         LR          0.544$\pm$0.128  0.879$\pm$0.029   
                         DCT         0.547$\pm$0.244  0.899$\pm$0.028   
30%          F-1 score   SVM         0.445$\pm$0.184  0.901$\pm$0.025   
                         LR          0.667$\pm$0.171  0.904$\pm$0.027   
                         DCT           0.501$\pm$0.2     0.9$\pm$0.03   
             sensitivity SVM         0.382$\pm$0.178  0.886$\pm$0.032   
                         LR          0.605$\pm$0.181   0.89$\pm$0.033   
                         DCT           0.437$\pm$0.2  0.897$\pm$0.029   
40%          F-1 score   SVM         0.508$\pm$0.181  0.881$\pm$0.031   
                         LR          0.694$\pm$0.209  0.883$\pm$0.031   
                         DCT         0.534$\pm$0.191  0.859$\pm$0.046   
             sensitivity SVM          0.443$\pm$0.17  0.866$\pm$0.035   
                         LR          0.651$\pm$0.217  0.866$\pm$0.039   
                         DCT          0.473$\pm$0.19  0.862$\pm$0.037   

                                               SmtNN           SVMSmt  \
missing rate metric      classifier                                     
0%           F-1 score   SVM         0.908$\pm$0.015   0.92$\pm$0.013   
                         LR          0.899$\pm$0.011  0.928$\pm$0.012   
                         DCT         0.901$\pm$0.024  0.906$\pm$0.025   
             sensitivity SVM         0.899$\pm$0.018  0.914$\pm$0.014   
                         LR          0.886$\pm$0.015  0.923$\pm$0.014   
                         DCT         0.894$\pm$0.023   0.908$\pm$0.02   
10%          F-1 score   SVM         0.898$\pm$0.024   0.92$\pm$0.029   
                         LR           0.89$\pm$0.019   0.919$\pm$0.02   
                         DCT          0.891$\pm$0.02  0.893$\pm$0.021   
             sensitivity SVM         0.881$\pm$0.033   0.91$\pm$0.038   
                         LR          0.872$\pm$0.028   0.91$\pm$0.027   
                         DCT         0.878$\pm$0.025   0.89$\pm$0.022   
20%          F-1 score   SVM         0.888$\pm$0.024  0.906$\pm$0.021   
                         LR          0.876$\pm$0.026  0.906$\pm$0.019   
                         DCT         0.884$\pm$0.021  0.903$\pm$0.022   
             sensitivity SVM          0.868$\pm$0.03  0.893$\pm$0.026   
                         LR          0.853$\pm$0.034  0.893$\pm$0.023   
                         DCT         0.867$\pm$0.024  0.899$\pm$0.022   
30%          F-1 score   SVM         0.883$\pm$0.026  0.908$\pm$0.023   
                         LR          0.869$\pm$0.033  0.904$\pm$0.021   
                         DCT         0.885$\pm$0.031  0.894$\pm$0.035   
             sensitivity SVM

In [6]:
concat_to_export = pd.concat((res_all[0][0][0][0], res_all[0][0][0][1], 
                        res_all[1][0][0][0], res_all[1][0][0][1], 
                        res_all[2][0][0][0], res_all[2][0][0][1], 
                        res_all[3][0][0][0], res_all[3][0][0][1], 
                        res_all[4][0][0][0], res_all[4][0][0][1]))
concat_to_export.to_csv("mean_f1_sensi_ecoli_low.csv", index = False)

In [7]:
print(all_res.to_latex(index = True, 
                       formatters = {"name": str.upper},
                      float_format = "{.1f}".format))

\begin{tabular}{lllllllll}
\toprule
    &             &     &         NearMiss &         kmeanSmt &            SmtNN &           SVMSmt &          InsHard &            OsImp \\
missing rate & metric & classifier &                  &                  &                  &                  &                  &                  \\
\midrule
0\% & F-1 score & SVM &    0.54\$\textbackslash pm\$0.17 &  0.916\$\textbackslash pm\$0.013 &  0.908\$\textbackslash pm\$0.015 &   0.92\$\textbackslash pm\$0.013 &  0.854\$\textbackslash pm\$0.023 &  0.904\$\textbackslash pm\$0.012 \\
    &             & LR &  0.756\$\textbackslash pm\$0.104 &   0.92\$\textbackslash pm\$0.017 &  0.899\$\textbackslash pm\$0.011 &  0.928\$\textbackslash pm\$0.012 &  0.852\$\textbackslash pm\$0.022 &  0.915\$\textbackslash pm\$0.012 \\
    &             & DCT &  0.768\$\textbackslash pm\$0.115 &   0.899\$\textbackslash pm\$0.03 &  0.901\$\textbackslash pm\$0.024 &  0.906\$\textbackslash pm\$0.025 &  0.836\$\textbackslash pm